<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# we upload, in case that we cant parse logs(a lot of time and ram consumption)

# Mount Google Drive to save datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#copy log file
!cp '/content/drive/MyDrive/zHDFS2/logs_train' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_ntest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/logs_atest' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/log2index' '/content/'
!cp '/content/drive/MyDrive/zHDFS2/reduced_embeddings' '/content/'


In [7]:
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import numpy as np


# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
with open('/content/reduced_embeddings', 'r') as json_file:
    embeddings = json.load(json_file)

In [5]:
len(embeddings[0])


10

In [6]:

name = 'logs_train'
window_size = 10
num_sessions = 0
inputs = []
#outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [ embeddings[int(i)] for i in row.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                #outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
#dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float))

Number of sessions(logs_train): 446578
Number of seqs(logs_train): 4704000


In [12]:
# Define the LSTM Autoencoder model with dropout
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_prob):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.decoder = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True, dropout=dropout_prob)


    def forward(self, x):
        encoded, _ = self.encoder(x)
        encoded = encoded[:,-1:,:] # output of last cell
        input_decode = torch.tile(encoded, (1, 10, 1))
        decoded, _ = self.decoder(input_decode)
        return decoded

In [13]:
model = LSTMAutoencoder(10, 16, 2, 0.2).to(device)
model

LSTMAutoencoder(
  (encoder): LSTM(10, 16, num_layers=2, batch_first=True, dropout=0.2)
  (decoder): LSTM(16, 10, num_layers=2, batch_first=True, dropout=0.2)
)

In [15]:
data = torch.randn(1,10, 10)
model(data.to(device)).shape

torch.Size([1, 10, 10])

In [16]:
# Calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters: {num_params}")

Number of Parameters: 5968


In [17]:
dataloader = DataLoader(dataset, batch_size=128)

# Loss and optimizer
criterion = nn.MSELoss()

In [18]:
len(dataloader) # regard to batch size

36750

In [19]:
for step, (seq) in enumerate(dataloader):
  print(seq[0].shape)
  break

torch.Size([128, 10, 10])


In [21]:
def adjust_learning_rate(optimizer, epoch, lr_step=(15,25,35,45), lr_decay_ratio=0.5):
    """Adjust the learning rate based on the epoch number."""
    if epoch == 0:
        optimizer.param_groups[0]['lr'] /= 16
    elif epoch in [1, 2, 3, 4]:  # in step five , we finish warm up ,and start normal learning rate
        optimizer.param_groups[0]['lr'] *= 2
    if epoch in lr_step: # in these steps , we are geting close to optimal point so we need to have shorter step
        optimizer.param_groups[0]['lr'] *= lr_decay_ratio
    return optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [23]:

num_epochs = 50
input_size = 10 # embedding size

# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    optimizer = adjust_learning_rate(optimizer, epoch)
    train_loss = 0
    for step, (seq) in enumerate(dataloader):
        # Forward pass
        seq = seq[0].clone().detach().view(-1, window_size, input_size).to(device)
        output = model(seq)
        loss = criterion(output, seq.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
print('Finished Training')


Epoch [1/50], train_loss: 0.0113
Epoch [2/50], train_loss: 0.0107
Epoch [3/50], train_loss: 0.0098
Epoch [4/50], train_loss: 0.0084
Epoch [5/50], train_loss: 0.0070
Epoch [6/50], train_loss: 0.0063
Epoch [7/50], train_loss: 0.0058


KeyboardInterrupt: ignored

In [24]:
torch.save(model.state_dict(), '/content/drive/MyDrive/zHDFS2/Autoencoder2_parameters.pth')

In [29]:
len(embeddings)

52

In [30]:
embeddings.append([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [31]:
def generate(name):
    window_size = 10
    hdfs = {} #store the unique sequences and their counts.
    length = 0
    with open('/content/' + name, 'r') as f:
        for row in f:
            line = [int(i) for i in row.strip().split()]
            line = line + [52] * (window_size + 1 - len(line)) #if the length of the line is less than windows size, it covers by 30 a template with zeros vector
            hdfs[tuple(line)] = hdfs.get(tuple(line), 0) + 1   #If the tuple is not present in the dictionary, hdfs.get(tuple(ln), 0) returns 0, and the code initializes the count to 1.
            length += 1
            # hdfs.append(tuple(line))
    print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return hdfs, length

In [32]:
test_normal_loader, test_normal_length = generate('logs_ntest')
test_abnormal_loader, test_abnormal_length = generate('logs_atest')

Number of sessions(logs_ntest): 1091
Number of sessions(logs_atest): 4126


In [33]:
def evaluation(threshold):
  # Test the model
  model.eval()

  TP = 0
  FP = 0

  start_time = time.time()
  with torch.no_grad():
      for line in test_normal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                FP += test_normal_loader[line] # numbers of that set we have
                break
  with torch.no_grad():
      for line in test_abnormal_loader.keys():
          for i in range(len(line) - window_size):
              session = line[i:i + window_size]
              seq = [embeddings[temp] for temp in session]
              seq = torch.tensor(seq, dtype=torch.float).view(-1, window_size, input_size).to(device)
              output = model(seq)

              loss = criterion(output, seq)

              if (loss.cpu().detach().numpy()>threshold):
                TP += test_abnormal_loader[line]
                break
  elapsed_time = time.time() - start_time
  print('elapsed_time: {:.3f}s'.format(elapsed_time))
  # Compute precision, recall and F1-measure
  FN = test_abnormal_length - TP
  P = 100 * TP / (TP + FP)
  R = 100 * TP / (TP + FN)
  F1 = 2 * P * R / (P + R)
  print('false positive (FP): {}, false negative (FN): {}, Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%'.format(FP, FN, P, R, F1))
  print('Finished Predicting')

In [34]:
threshold = [0.005,0.01,0.015,0.02,0.025,0.03]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ', i)
  evaluation(i)

elapsed_time: 11.347s
false positive (FP): 82525, false negative (FN): 10, Precision: 16.938%, Recall: 99.941%, F1-measure: 28.966%
Finished Predicting
elapsed_time: 16.289s
false positive (FP): 21342, false negative (FN): 235, Precision: 43.755%, Recall: 98.604%, F1-measure: 60.614%
Finished Predicting
elapsed_time: 25.766s
false positive (FP): 8193, false negative (FN): 3514, Precision: 61.923%, Recall: 79.131%, F1-measure: 69.477%
Finished Predicting
elapsed_time: 38.531s
false positive (FP): 3853, false negative (FN): 5945, Precision: 73.871%, Recall: 64.693%, F1-measure: 68.978%
Finished Predicting
elapsed_time: 47.681s
false positive (FP): 592, false negative (FN): 8432, Precision: 93.421%, Recall: 49.923%, F1-measure: 65.072%
Finished Predicting
elapsed_time: 49.056s
false positive (FP): 486, false negative (FN): 11512, Precision: 91.638%, Recall: 31.631%, F1-measure: 47.029%
Finished Predicting


In [ ]:
threshold = [0.006,0.007,0.008,0.009,0.01,0.011,0.12,0.13]
for i in threshold:
  print('-------------------------------------------------------------------------')
  print('threshold = ',i)
  evaluation(i)

-------------------------------------------------------------------------
threshold =  0.006
elapsed_time: 17.042s
false positive (FP): 6025, false negative (FN): 215, Precision: 73.397%, Recall: 98.723%, F1-measure: 84.197%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.007
elapsed_time: 18.647s
false positive (FP): 5008, false negative (FN): 1006, Precision: 75.969%, Recall: 94.025%, F1-measure: 84.038%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.008
elapsed_time: 19.471s
false positive (FP): 4154, false negative (FN): 1087, Precision: 79.131%, Recall: 93.544%, F1-measure: 85.736%
Finished Predicting
-------------------------------------------------------------------------
threshold =  0.009
elapsed_time: 23.250s
false positive (FP): 1805, false negative (FN): 1270, Precision: 89.610%, Recall: 92.458%, F1-measure: 91.012%
Finished Predicting
-----------------

ZeroDivisionError: ignored